In [2]:
!pip install langchain

In [3]:
!pip install openai

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [5]:
import sqlite3
print(sqlite3.sqlite_version)

3.37.2


In [13]:
import os
import sqlite3
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize SQLite database to log mistakes
def init_db():
    conn = sqlite3.connect('mistakes.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mistakes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_input TEXT,
            correction TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Insert mistakes into DB
def log_mistake(user_input, correction):
    conn = sqlite3.connect('mistakes.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO mistakes (user_input, correction) VALUES (?, ?)", (user_input, correction))
    conn.commit()
    conn.close()

# Fetch all mistakes for review
def fetch_mistakes():
    conn = sqlite3.connect('mistakes.db')
    cursor = conn.cursor()
    cursor.execute("SELECT user_input, correction FROM mistakes")
    data = cursor.fetchall()
    conn.close()
    return data

# Main chatbot function
def start_chat():
    init_db()

    # Set your OpenAI API Key
    os.environ["OPENAI_API_KEY"] = 'sk-abc123XYZyourREALkey'

    # Initialize LangChain components
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=llm, memory=memory)

    # Collect user preferences
    chatbot = input("Hi! User:")
    chatbot = print("Welcome to the Language Tutor Chatbot!")
    chatbot = input("What is your name? ")
    chatbot = print(f"Nice to meet you, {chatbot}!")
    chatbot = print("How may I help you")
    user = input("User: ")
    known_lang = input("Which language do you know? ")
    target_lang = input("Which language do you want to learn? ")
    level = input("What is your current level? (Beginner / Intermediate / Advanced): ")
    scene = input("Choose a scene (e.g., Restaurant, Airport, Market, Mall, Metro, Subways, House): ")

    print(f"\nStarting conversation in {target_lang} (Scene: {scene})...\nType 'exit' to finish.\n")

    # Chat loop
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break

        # Get response from AI
        prompt = (f"You are a {target_lang} language tutor. Correct user if needed. "
                  f"User knows {known_lang} and is learning {target_lang} at {level} level. "
                  f"Scene: {scene}. Help them improve.\nUser: {user_input}")
        response = conversation.predict(input=prompt)
        print(f"Bot: {response}\n")

        # Basic mistake detection (Simulated Logic - Customize this)
        if "wrong" in user_input.lower() or "incorrect" in user_input.lower():
            correction = f"Suggested correction for: '{user_input}' -> [Provide correct phrase here]"
            log_mistake(user_input, correction)
            print(f"Hint: {correction}\n")

    # Show mistakes review
    mistakes = fetch_mistakes()
    if mistakes:
        print("\n Mistake Summary & Review ")
        for idx, (user_input, correction) in enumerate(mistakes, start=1):
            print(f"{idx}. Mistake: '{user_input}' | Correction: {correction}")
        print("\nFocus on grammar and sentence construction to improve next time.")
    else:
        print("Well done! You made no tracked mistakes this session.")

if __name__ == "__main__":
    start_chat()


Hi! User:Hi
Welcome to the Language Tutor Chatbot!
What is your name? Yash
Nice to meet you, Yash!
How may I help you
User: I want to learn a new language
Which language do you know? Hindi
Which language do you want to learn? English
What is your current level? (Beginner / Intermediate / Advanced): Beginner
Choose a scene (e.g., Restaurant, Airport, Market, Mall, Metro, Subways, House): Airport 

Starting conversation in English (Scene: Airport )...
Type 'exit' to finish.

You: exit
Well done! You made no tracked mistakes this session.
